# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking current working directory
#print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
# print(filepath)

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(filepath,'*'))
#     print(file_path_list)
# print(len(file_path_list))

/home/workspace/event_data
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [110]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
print(len(file_path_list))
try:    
    # for every filepath in the file path list 
    for f in file_path_list:
        print(f)
    # reading csv file 
        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            # creating a csv reader object 
            csvreader = csv.reader(csvfile) 
            next(csvreader)

     # extracting each data row one by one and append it        
            for line in csvreader:
                #print(line)
                full_data_rows_list.append(line) 

    # uncomment the code below if you would like to get total number of rows 
    # print(len(full_data_rows_list))
    # uncomment the code below if you would like to check to see what the list of event data rows will look like
    # print(full_data_rows_list)

    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
except Exception as e:
    print(e)

30
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-03-events.csv
/home/workspace/event_data/2018-11-13-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-17-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-08-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/e

In [111]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
# Create a KEYSPACE
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify_cassandra_db
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
    print("KEYSPACE sparkify_cassandra_db is Ready to use")
except Exception as e:
    print (e)

KEYSPACE sparkify_cassandra_db is Ready to use


In [6]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify_cassandra_db')
    print("session of sparkify_cassandra_db is ready")
except Exception as e:
    print(e)

session of sparkify_cassandra_db is ready


## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Question 1: 
#### Give me the artist, song title and song's length in the music app history
#### that was heard during sessionId = 338, and itemInSession = 4

### Query Description: 
###### In this query, I used 'sessionId' as the partition key and 'itemInSession' as my clustering key.
###### Each partition is uniquely identified by 'sessionId' 
###### while 'itemInSession' was used to uniquely identify the rows within a partition to sort the data by the value of 'itemInSession'.

In [7]:
# Question 1. (CREATE TABLE)

try:
    drop_table = "DROP TABLE IF EXISTS music_app_hist_by_session_item"
    session.execute(drop_table)
    create_table = """
        CREATE TABLE IF NOT EXISTS music_app_hist_by_session_item (
            sessionId int,
            itemInSession int,
            artist text,
            length float,
            songTitle text,
            PRIMARY KEY(sessionId, itemInSession)
        )
        """
    session.execute(create_table)
    print("Table 'music_app_hist_by_session_item' is created")
except Exception as e:
    print(e)

                    

Table 'music_app_hist_by_session_item' is created


In [8]:
# Question 1. (Insert Data)
# Parse the csv file and insert the data to the table
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO music_app_hist_by_session_item (sessionId, itemInSession, artist, length, songTitle)"
            query = query + "VALUES(%s, %s, %s, %s, %s)"
            session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
    print("song info is inserted into 'music_app_hist_by_session_item'")
except Exception as e:
    print(e)

song info is inserted into 'music_app_hist_by_session_item'


#### Do a SELECT to verify that the data have been inserted into each table

In [9]:
# Question 1. (Query Data)
# Query the data from the table and print to verify

# query = "SELECT artist, songTitle, length from music_app_hist_by_session_item WHERE sessionId = 338 and itemInSession = 4"
query = "SELECT artist, songTitle, length from music_app_hist_by_session_item WHERE sessionId = %s and itemInSession = %s"
prettyPrint = PrettyTable()
prettyPrint.field_names = ["Artist Name", "Song Title", "Song Length"]
try:
#     rows = session.execute(query)
    rows = session.execute(query, (338, 4, ))
    for row in rows:
        #print(row)
        prettyPrint.add_row([row.artist, row.songtitle, row.length])
        #print (row.artist, row.songtitle, row.length)
    print(prettyPrint)
except Exception as e:
    print(e)

+-------------+---------------------------------+--------------------+
| Artist Name |            Song Title           |    Song Length     |
+-------------+---------------------------------+--------------------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-------------+---------------------------------+--------------------+


### Question 2: 
####     Give me only the following: 
####     name of artist, song (sorted by itemInSession) and user (first and last name)
####     for userid = 10, sessionid = 182


### Query Description: 
######     In this query, I used '*userid*' and '*sessionId*' as the composite partition key for performance issue
######     and '*itemInSession*' as the clustering key so that result would be sorted by it.
######     Each partition is uniquely identified by *itemInSession* while itemInSession was used to uniquely identify the rows within a partition to sort the data by the value of *itemInSession*.

In [34]:
# Question 2

# Step 1: Drop and Create Table
try:
    drop_table = "DROP TABLE IF EXISTS song_list_by_user_session"
    session.execute(drop_table)
    create_table = """
        CREATE TABLE IF NOT EXISTS song_list_by_user_session (
            userId int,
            sessionId int,
            itemInSession int,
            artist text,
            firstName text,
            lastName text,
            songTitle text,
            PRIMARY KEY((userid, sessionId), itemInSession)
        )
        """
    session.execute(create_table)
    print("Step 1: Table 'song_list_by_user_session' is created")
except Exception as e:
    print(e)

# Step 2: Insert INTO TABLE
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        print("Step 2: Inserting 'song_list_by_user_session'...")
        for line in csvreader:
            ## Assign the INSERT statements into the `query` variable
            query = "INSERT INTO song_list_by_user_session (userId, sessionId, itemInSession, artist, firstName, lastName, songTitle)"
            query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        print("Step 2: song info is inserted into 'song_list_by_user_session'")
except Exception as e:
    print(e)

# Step 3: Execute Select Statement
try:
    print("Step 3: Query and Verify data in 'song_list_by_user_session'")
    query = "SELECT artist, songTitle, firstName, lastName from song_list_by_user_session WHERE userid = %s and sessionid = %s;"
    prettyPrint = PrettyTable()
    prettyPrint.field_names = ["Artist Name", "Song Title", "First Name", "Last Name"]
    rows = session.execute(query, (10, 182, ))
    for row in rows:
        #print (row.artist, row.songtitle, row.firstname, row.lastname)
        prettyPrint.add_row([row.artist, row.songtitle, row.firstname, row.lastname])
    print(prettyPrint)
except Exception as e:
    print(e)

Step 1: Table 'song_list_by_user_session' is created
Step 2: Inserting 'song_list_by_user_session'...
Step 2: song info is inserted into 'song_list_by_user_session'
Step 3: Query and Verify data in 'song_list_by_user_session'
+-------------------+------------------------------------------------------+------------+-----------+
|    Artist Name    |                      Song Title                      | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+---------------------------------------

### Question 3: 
#### Give me every user name (first and last) in my music app history
#### who listened to the song 'All Hands Against His Own'

### Query Description: 
###### In this query, I used *'songTitle'* as the partition key and '*user_id*' as my clustering key.
###### Each partition is uniquely identified by '*songTitle*' 
###### while '*user_id*' was used to uniquely identify the rows within a partition to sort the data by the value of '*user_id*'.

In [42]:

# Step 1: CREATE TABLE
try:
    drop_table = "DROP TABLE IF EXISTS user_list_by_song_title"
    session.execute(drop_table)
    create_table = """
        CREATE TABLE IF NOT EXISTS user_list_by_song_title (
            songTitle text,
            user_id int, 
            firstName text,
            lastName text,
            PRIMARY KEY ((songTitle), user_id)
        )
        """
    session.execute(create_table)
    print("Step 1: Table 'user_list_by_song_title' is created")
except Exception as e:
    print(e)

# Step 2: Insert INTO TABLE
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        print("Step 2: Inserting 'user_list_by_song_title'...")
        for line in csvreader:
            ## Assign the INSERT statements into the `query` variable
            query = "INSERT INTO user_list_by_song_title (songTitle, user_id, firstName, lastName)"
            query = query + "VALUES(%s, %s, %s, %s)"
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        print("Step 2: user info is inserted into 'user_list_by_song_title'")
except Exception as e:
    print(e)

# Step 3: Execute Select Statement
try:
    print("Step 3: Query and Verify data in 'user_list_by_song_title'")
    query = "SELECT firstName, lastName from user_list_by_song_title WHERE songtitle = %s"
    prettyPrint = PrettyTable()
    prettyPrint.field_names = ["First Name", "Last Name"]
    rows = session.execute(query, ('All Hands Against His Own', ))
    for row in rows:
        #print (row.firstname, row.lastname)
        prettyPrint.add_row([row.firstname, row.lastname])
    print(prettyPrint)
except Exception as e:
    print(e)

Step 1: Table 'user_list_by_song_title' is created
Step 2: Inserting 'user_list_by_song_title'...
Step 2: user info is inserted into 'user_list_by_song_title'
Step 3: Query and Verify data in 'user_list_by_song_title'
+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [14]:
# Drop the table before closing out the sessions

try:
    drop_applist = "DROP TABLE IF EXISTS music_app_hist_by_session_item"
    session.execute(drop_applist)
    print("DROP TABLE music_app_hist_by_session_item")

    drop_songlist = "DROP TABLE IF EXISTS song_list_by_user_session"
    session.execute(drop_songlist)
    print("DROP TABLE song_list_by_user_session")
    
    drop_userlist = "DROP TABLE IF EXISTS user_list_by_song_title"
    session.execute(drop_userlist)
    print("DROP TABLE user_list_by_song_title")
except Exception as e:
    print(e)

In [ ]:
# Drop keyspace
try:
    drop_keyspace = "DROP KEYSPACE IF EXISTS sparkify_cassandra_db"
    session.execute(drop_keyspace)
    print("KEYSPACE sparkify_cassandra_db is DROPPED")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [15]:
session.shutdown()
cluster.shutdown()